## Dependency loading, initial processing

In [1]:
!pip install datasets

import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from datasets import load_dataset
from tqdm import tqdm

In [2]:
# Reproducibility seeding
def set_seed(seed=100):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(777)

In [3]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Load HateXplain dataset from Hugging Face
print("Loading the HateXplain dataset...")
try:
    dataset = load_dataset("hatexplain")
    print("Dataset loaded successfully")
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise

Loading the HateXplain dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset loaded successfully


In [5]:
# Basic information about the dataset structure
print("\nDataset structure:")
print(f"Train set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")

# Example item (commented-out to avoid senstive example on github):
print("\nSample item from training set:")
sample_item = dataset['train'][0]
print(f"Keys: {sample_item.keys()}")
print(f"Annotators keys: {sample_item['annotators'].keys()}")
print(f"Label structure: {sample_item['annotators']['label']}")
#print(f"Sample text: {' '.join(sample_item['post_tokens'])[:100]}")


Dataset structure:
Train set size: 15383
Validation set size: 1922
Test set size: 1924

Sample item from training set:
Keys: dict_keys(['id', 'annotators', 'rationales', 'post_tokens'])
Annotators keys: dict_keys(['label', 'annotator_id', 'target'])
Label structure: [0, 2, 2]


## Utility Functions

In [6]:
# Define the preprocessing function
def preprocess_data(dataset_split):
    """
    Process the HateXplain dataset split and extract texts and labels.
    In HateXplain, labels are already numeric:
    hatespeech (0), normal (1) or offensive (2)
    """
    texts = []
    labels = []

    for item in tqdm(dataset_split, desc="Processing data"):
        # Extract text from tokens
        post_tokens = item['post_tokens']
        text = " ".join(post_tokens)

        # Get the annotations
        annotator_labels = item['annotators']['label']

        # Compute majority vote for the label
        if len(annotator_labels) > 0:
            # Count occurrences of each label
            label_counts = {}
            for label in annotator_labels:
                if label in label_counts:
                    label_counts[label] += 1
                else:
                    label_counts[label] = 1

            # Find the majority label
            majority_label = max(label_counts.items(), key=lambda x: x[1])[0]

            # Add to dataset
            texts.append(text)
            labels.append(majority_label)

    return texts, labels

# Create a custom dataset class
class HateXplainDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Define training function
def train():
    # Initialize variables to track best model
    best_val_loss = float('inf')
    best_model_path = 'best_bert_hatexplain_model.pt'

    print(f"\nStarting training for {epochs} epochs...")

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch in progress_bar:
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            model.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass and optimization
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})

        avg_train_loss = train_loss / len(train_dataloader)

        # Validation
        model.eval()
        val_loss = 0
        val_preds = []
        val_true = []

        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validation"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                val_loss += loss.item()

                preds = torch.argmax(logits, dim=1).cpu().numpy()
                true_labels = labels.cpu().numpy()

                val_preds.extend(preds)
                val_true.extend(true_labels)

        avg_val_loss = val_loss / len(val_dataloader)
        val_accuracy = accuracy_score(val_true, val_preds)
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(
            val_true, val_preds, average='weighted'
        )

        # Print metrics
        print(f"\nEpoch {epoch+1}/{epochs}")
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Val Loss: {avg_val_loss:.4f}")
        print(f"Val Accuracy: {val_accuracy:.4f}")
        print(f"Val F1 Score: {val_f1:.4f}")

        # Save the best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved best model to {best_model_path}")

    return best_model_path

# Function to evaluate model on test set
def evaluate(model_path):
    # Load the best model
    print(f"\nLoading best model from {model_path}...")
    model.load_state_dict(torch.load(model_path))
    model.eval()

    test_preds = []
    test_true = []

    print("Evaluating on test set...")
    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            true_labels = labels.cpu().numpy()

            test_preds.extend(preds)
            test_true.extend(true_labels)

    # Calculate metrics
    accuracy = accuracy_score(test_true, test_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        test_true, test_preds, average='weighted'
    )

    # Generate classification report
    report = classification_report(
        test_true,
        test_preds,
        target_names=['hate', 'normal', 'offensive'],
        output_dict=True
    )

    print("\nTest Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nDetailed Classification Report:")
    print(pd.DataFrame(report).transpose())

    # Error analysis
    error_analysis(test_texts, test_true, test_preds)

    return accuracy, precision, recall, f1, report


# Function for error analysis
def error_analysis(texts, true_labels, pred_labels):
    errors = []
    for i in range(len(texts)):
        if true_labels[i] != pred_labels[i]:
            errors.append({
                'text': texts[i],
                'true_label': ['hate', 'normal', 'offensive'][true_labels[i]],
                'pred_label': ['hate', 'normal', 'offensive'][pred_labels[i]]
            })

    # Save some examples of errors
    error_df = pd.DataFrame(errors)
    print(f"\nFound {len(errors)} misclassifications out of {len(texts)} samples")

    # Code below shows examples of mis-classifications
    # Commented-out to avoid senstive information when viewing notebook
    # if len(errors) > 0:
    #     print("\nSample of misclassifications:")
    #     sample_size = min(5, len(errors))
    #     for i in range(sample_size):
    #         print(f"\nExample {i+1}:")
    #         print(f"Text: {error_df.iloc[i]['text'][:100]}...")
    #         print(f"True label: {error_df.iloc[i]['true_label']}")
    #         print(f"Predicted label: {error_df.iloc[i]['pred_label']}")

    # Save error analysis to CSV
    error_df.to_csv('misclassifications.csv', index=False)
    print("\nSaved misclassifications to 'misclassifications.csv'")

## Driver Cell

In [7]:
# Process the dataset
print("\nProcessing datasets...")
train_texts, train_labels = preprocess_data(dataset['train'])
val_texts, val_labels = preprocess_data(dataset['validation'])
test_texts, test_labels = preprocess_data(dataset['test'])

# Verify dataset sizes
print(f"\nProcessed dataset sizes:")
print(f"Train samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")
print(f"Test samples: {len(test_texts)}")

# Check label distribution
train_label_dist = pd.Series(train_labels).value_counts(normalize=True)
print("\nTraining label distribution:")
print(train_label_dist)
print("Label meanings: 0 = hatespeech, 1 = normal, 2 = offensive")

# Initialize tokenizer and model
print("\nInitializing BERT model and tokenizer...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False
)

# Move model to device
model.to(device)

# Create datasets
print("Creating PyTorch datasets...")
train_dataset = HateXplainDataset(train_texts, train_labels, tokenizer)
val_dataset = HateXplainDataset(val_texts, val_labels, tokenizer)
test_dataset = HateXplainDataset(test_texts, test_labels, tokenizer)

print(f"Dataset sizes:")
print(f"Train: {len(train_dataset)}")
print(f"Validation: {len(val_dataset)}")
print(f"Test: {len(test_dataset)}")

# Create dataloaders
batch_size = 32
print("\nCreating dataloaders...")
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Training hyperparameters
print("\nSetting up training parameters...")
epochs = 4
learning_rate = 2e-5
warmup_steps = 0
weight_decay = 0.01

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

try:
    print("\nStarting fine-tuning process...")
    best_model_path = train()
    print("\nEvaluating model on test set...")
    accuracy, precision, recall, f1, report = evaluate(best_model_path)

    print("\nFinished model tuning and evaluation")
except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()


Processing datasets...


Processing data: 100%|██████████| 1924/1924 [00:00<00:00, 3428.01it/s]



Processed dataset sizes:
Train samples: 15383
Validation samples: 1922
Test samples: 1924

Training label distribution:
1    0.406358
0    0.308652
2    0.284990
Name: proportion, dtype: float64
Label meanings: 0 = hatespeech, 1 = normal, 2 = offensive

Initializing BERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating PyTorch datasets...
Dataset sizes:
Train: 15383
Validation: 1922
Test: 1924

Creating dataloaders...

Setting up training parameters...

Starting fine-tuning process...

Starting training for 4 epochs...


...


Epoch 3/4
Train Loss: 0.5120
Val Loss: 0.7610
Val Accuracy: 0.6805
Val F1 Score: 0.6713


Validation: 100%|██████████| 61/61 [00:31<00:00,  1.94it/s]
<ipython-input-6-eb3caf478545>:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.loa


Epoch 4/4
Train Loss: 0.4082
Val Loss: 0.8235
Val Accuracy: 0.6790
Val F1 Score: 0.6757

Evaluating model on test set...

Loading best model from best_bert_hatexplain_model.pt...
Evaluating on test set...


Testing: 100%|██████████| 61/61 [00:31<00:00,  1.93it/s]


Test Results:
Accuracy: 0.6944
Precision: 0.6839
Recall: 0.6944
F1 Score: 0.6825

Detailed Classification Report:
              precision    recall  f1-score      support
hate           0.733846  0.803030  0.766881   594.000000
normal         0.707263  0.809463  0.754919   782.000000
offensive      0.596306  0.412409  0.487594   548.000000
accuracy       0.694387  0.694387  0.694387     0.694387
macro avg      0.679138  0.674967  0.669798  1924.000000
weighted avg   0.683867  0.694387  0.682472  1924.000000

Found 588 misclassifications out of 1924 samples

Saved misclassifications to 'misclassifications.csv'

Finished model tuning and evaluation
